### Preparing AI Ready Data
This notebook extracts statistics from 100 days of seismic data for use in classic machine learning. The environment requires obspy to be installed in addition to the standard mlgeo environment

In [52]:
#!pip install obspy
import obspy
from obspy import read
from obspy import UTCDateTime as utc
from scipy.signal import stft
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
#Make folder for AI ready data
! mkdir data/ai_ready
filepath = os.getcwd() + '/data/ai_ready/'
clean_filepath = os.getcwd() + '/data/clean/'
raw_filepath = os.getcwd() + '/data/raw/'

mkdir: data/ai_ready: File exists


In [24]:
#set eruption time for given sensor/volcano, example is for Sitkin Volcano
eruption_time = utc('2021-05-26T05:04')

In [84]:
#read in dates file from Download_Data notebook and convert to UTC objects
dates = [utc(date) for date in np.load(raw_filepath+'date_list.npy', allow_pickle=True)]
print(dates[0])

2021-04-06T00:00:00.000000Z


### Define Functions for Calculating Stats

In [57]:
# General Stats Used
def stats(data):
    mean = np.mean(data)
    maxx = np.max(data)
    minn = np.min(data)
    median = np.median(data)
    range = np.abs(maxx - minn)
    std = np.std(data)
    var = np.var(data)

    return(mean, maxx, minn, median, range, std, var)

# Spectral Domain, use STFT using the same parameters as Zahra et al. 2024
def compute_spec(data):

    x = data
    fs = 8 # specified by Zahra
    nwindow = 256 #given by Zahra
    noverlap = 32 #given by Zahra

    f, t, spectra = stft(x, fs=fs, nperseg=nwindow, noverlap=noverlap)

    #find magnitude of complex spectra
    spectra = np.abs(spectra)

    #find log 10 of spectra
    log_spectra = np.log10(spectra)



    return(spectra, log_spectra)

### Cell Below Will Save Seismograms in Day Chunks and also Combine Them into 1 Large File

In [12]:
#read in cleaned mseed files, save as daily npy files and one giant 100 day file
hundred_data = np.array([])

for i in range(100):
    i += 1

    #read mseed
    st = read(clean_filepath+str(i)+'_cleaned.mseed')

    #save data to day long npys
    data = st[0].data 
    np.save(filepath+str(i)+'_ready.npy', data)

    #append to overall array
    hundred_data = np.append(hundred_data, data)

#save hundred day data npy
np.save(filepath+'100_days.npy', hundred_data)

### Cells Below Will Compute and Save Time Series and Spectral Statistics For Each Hour of Seismogram Data
Data will also be associated with a Before or After Eruption Label in Stats file

In [85]:
#initialize statistics dataframe
statistics = ['mean','max','min','median','range','std','var',
              'spec_mean','spec_max','spec_min','spec_median','spec_range','spec_std','spec_var',
              'speclog_mean','speclog_max','speclog_min','speclog_median','speclog_range','speclog_std','speclog_var',
              'state']

stats_df = pd.DataFrame(columns=statistics)

In [86]:
for i in range(100):
    date = dates[i]
    date += 30*60 #add half an hour for determining eruption state
    i += 1

    #read mseed
    st = read(clean_filepath+str(i)+'_cleaned.mseed')
    data = st[0].data
    hz = st[0].stats.sampling_rate

    #THIS ASSUMES DATA CONTAINS 24 HOURS OF DATA PER DATA
    split_data = np.array(np.split(data, 24))

    if split_data.shape[1] == 28800:
        for hour in np.arange(24):
            date += 60*60
            if date >= eruption_time:
                state = 'after'
            elif date < eruption_time:
                state = 'before'

            data = split_data[hour]
            spectra, log_spectra = compute_spec(data)

            tmean, tmax, tmin, tmedian, trange, tstd, tvar = stats(data)

            smean, smax, smin, smedian, srange, sstd, svar = stats(spectra)

            slmean, slmax, slmin, slmedian, slrange, slstd, slvar = stats(log_spectra)

            stats_df.loc[len(stats_df)] = tmean, tmax, tmin, tmedian, trange, tstd, tvar, smean, smax, \
                smin, smedian, srange, sstd, svar, slmean, slmax, slmin, slmedian, slrange, slstd, slvar, state


In [87]:
#inspect dataframe header
stats_df.head(3)

,mean,max,min,median,range,std,var,spec_mean,spec_max,spec_min,...,spec_std,spec_var,speclog_mean,speclog_max,speclog_min,speclog_median,speclog_range,speclog_std,speclog_var,state
0,-2.045739,1659.079530,-1318.335883,-6.963337,2977.415414,371.791134,138228.647271,10.392417,995.277247,0.004666,...,31.456352,989.502099,0.433941,2.997944,-2.331057,0.297929,5.329001,0.661548,0.437646,before
1,-43.245231,1800.918051,-1867.272921,-56.748376,3668.190972,476.115885,226686.335511,12.038669,1081.727088,0.010802,...,40.854324,1669.075788,0.473613,3.034118,-1.966483,0.329268,5.000601,0.658096,0.433090,before
2,-34.465366,2031.091640,-2252.112966,-32.267577,4283.204606,548.338617,300675.238931,12.984918,1236.088867,0.011445,...,49.401049,2440.463664,0.500578,3.092050,-1.941383,0.366436,5.033433,0.640550,0.410304,before


In [90]:
#save stats to csv file
stats_df.to_csv(path_or_buf=filepath+'statistics.csv', index=False)